In [ ]:
import random
from collections import deque
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create environment
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]  # 4
action_size = env.action_space.n
state_size, action_size


In [ ]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

## What is a Replay Buffer?

In Reinforcement Learning (RL), especially DQN, we use a Replay Buffer to store what the agent sees and does, and then train from that stored experience rather than only the latest step.

This makes learning more stable and efficient.

**Buffer_size**: Maximum number of experiences the buffer can hold (e.g., 10,000).

**Batch_size**: Number of experiences to pull out when it's time to learn (e.g., 64).

In [ ]:
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size  # <-- Add this line

    def add(self, state, action, reward, next_state, done):
        self.memory.append(
            (state, action, reward, next_state, done))  #This method adds a new experience into the buffer.

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        #This chunk is converting raw experience data into PyTorch tensors for training.
        states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float().to(device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float().to(device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.float32)).float().to(device)
        return (states, actions, rewards, next_states,
                dones)  #Returns a tuple of tensors that the agent can use to learn (Q-values).

    def __len__(self):  #This lets you call len(buffer) to check how many experiences are stored.
        return len(self.memory)

#Purpose: You're setting up the memory space and how big each learning bite will be.
# Why random?
#    To break correlation between steps.
#    Prevents overfitting to recent events.

## Class in Deep Q-Learning (DQN)
This class defines a DQN agent that interacts with the environment, stores experiences, and learns from them using two neural networks (local and target).

- Local network is used to choose actions.

- Target network is used to compute stable targets for learning.

**gamma** :	Discount factor for future rewards

**lr** :	Learning rate for optimizer

**tau** :	Factor for soft updating target network

### This is the ε-greedy strategy, a key part of balancing exploration and exploitation.

- random.random() generates a number between 0 and 1.

- If that number is greater than epsilon, the agent chooses the best action using the network.

- So the probability of choosing the best action is 1 - epsilon.

- Otherwise, it chooses a random action to explore.

In [ ]:
class Agent:
    def __init__(self, state_size, action_size, buffer_size=10000, batch_size=64, gamma=0.99, lr=1e-3, tau=1e-2):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma
        self.lr = lr
        self.tau = tau

        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=lr)
        self.memory = ReplayBuffer(buffer_size, batch_size)
        self.t_steps = 0
        self.update_every = 4  # Learn every 4 steps


    def act(self, state, epsilon=0.0):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)

        #Evaluate the model without training it (no gradient tracking).
        self.qnetwork_local.eval()
        with torch.no_grad():
            q_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        #With probability 1 - epsilon, take the best action. Otherwise, choose a random action (for exploration).
        if random.random() > epsilon:
            return np.argmax(q_values.cpu().data.numpy())
        else:
            return np.random.choice(np.arange(self.action_size))

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)
        self.t_steps = (self.t_steps + 1) % self.batch_size
        if self.t_steps % self.update_every == 0 and len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences

        #Use the target network to get the best Q-value for next state.
        #detach() to avoid gradients from flowing through target net
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        #Get Q-values from local net for the actions taken.
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.qnetwork_local, self.qnetwork_target)

    def soft_update(self, local_model, target_model):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)

    #This blends the parameters of the local model into the target model using soft updates
    # θ_target = τ * θ_local + (1 - τ) * θ_target


In [ ]:
n_episodes = 500
max_t = 1000
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
scores = []
scores_window = deque(maxlen=100)
agent = Agent(state_size, action_size)

In [ ]:
for episode in range(1, n_episodes + 1):
    state, _ = env.reset()
    total_reward = 0

    for t in range(max_t):
        action = agent.act(state, epsilon)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Modified reward to encourage longer episodes
        if not done:
            reward = 1.0
        else:
            reward = -10.0

        agent.step(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            break

    scores.append(total_reward)
    scores_window.append(total_reward)
    epsilon = max(epsilon_min, epsilon_decay * epsilon)

    print(f"Episode {episode}, Score: {total_reward}, Avg100: {np.mean(scores_window):.2f}, Epsilon: {epsilon:.2f}")

    if np.mean(scores_window) >= 195.0:
        print(f"\nEnvironment solved in {episode} episodes!")
        torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        break

env.close()

In [ ]:
import gymnasium as gym
import base64
import imageio
import os
from IPython.display import HTML, display

def show_video_of_model(agent, env_name="CartPole-v1", output_path="video.mp4"):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []

    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()

    # Save video
    imageio.mimsave(output_path, frames, fps=30)
    print(f"✅ Video saved to: {os.path.abspath(output_path)}")

def show_video(video_path="video.mp4"):
    if os.path.exists(video_path):
        with open(video_path, 'rb') as f:
            video = f.read()
        encoded = base64.b64encode(video).decode('ascii')
        display(HTML(f'''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>'''))
    else:
        print(f"❌ Could not find video at {video_path}")

# 🔽 Example usage:
output_file = "D:/Artificial/AI/cartpole_trained.mp4"
show_video_of_model(agent, env_name="CartPole-v1", output_path=output_file)
show_video(output_file)
